In [139]:
import pandas as pd
import requests

def get_data_from_api(query):
  """
  given an api endpoint, get data and return as a dataframe
  """
  r = requests.get(query)
  x = r.json()
  df = pd.DataFrame.from_dict(x['result']['records'])
  return df



#API returns only 32k; so import via csv link
#tests = get_data_from_api('https://data.nsw.gov.au/data/api/3/action/datastore_search_sql?sql=SELECT * from "fb95de01-ad82-4716-ab9a-e15cf2c78556"')

# Get cases data

In [140]:
cases = get_data_from_api('https://data.nsw.gov.au/data/api/3/action/datastore_search_sql?sql=SELECT * from "2776dbb8-f807-4fb2-b1ed-184a6fc2c8aa"')

cases = cases.groupby(['notification_date','lga_name19'])['postcode'].count().reset_index()
cases.rename(columns={'postcode':'num_cases'}, inplace=True)
cases.groupby('notification_date')['num_cases'].sum()
cases.head()

,notification_date,lga_name19,num_cases
0,2020-01-25,Burwood (A),1
1,2020-01-25,Ku-ring-gai (A),1
2,2020-01-25,Parramatta (C),1
3,2020-01-27,Randwick (C),1
4,2020-03-01,Fairfield (C),1


## Get tests data
the API limits the data to only 32k records, so need to import via csv:

In [141]:
#get tests data
import pandas as pd
import io
import requests
url="https://data.nsw.gov.au/data/datastore/dump/fb95de01-ad82-4716-ab9a-e15cf2c78556?bom=True"
s=requests.get(url).content
tests=pd.read_csv(io.StringIO(s.decode('utf-8')))

#aggregate such that have num cases per day per lga

tests = tests.groupby(['test_date', 'lga_name19'])['test_count'].sum().reset_index()
tests.head()

,test_date,lga_name19,test_count
0,2020-01-01,Blacktown (C),1
1,2020-01-01,Camden (A),2
2,2020-01-01,Canada Bay (A),2
3,2020-01-01,Canterbury-Bankstown (A),2
4,2020-01-01,Cumberland (A),1


In [142]:
print("check number of records in cases:")
print(cases.shape)
print()
print("check number of records in tests:")
print(tests.shape)
print()

check number of records in cases:
(2879, 3)

check number of records in tests:
(61763, 3)



# Merge the test data and cases data

In [143]:
cases_tests = tests.merge(cases, 
                          how='left', 
                          left_on=['test_date','lga_name19'],
                          right_on=['notification_date', 'lga_name19']
                          )

cases_tests

,test_date,lga_name19,test_count,notification_date,num_cases
0,2020-01-01,Blacktown (C),1,NaN,NaN
1,2020-01-01,Camden (A),2,NaN,NaN
2,2020-01-01,Canada Bay (A),2,NaN,NaN
3,2020-01-01,Canterbury-Bankstown (A),2,NaN,NaN
4,2020-01-01,Cumberland (A),1,NaN,NaN
...,...,...,...,...,...
61758,2021-07-23,Wingecarribee (A),353,2021-07-23,1.0
61759,2021-07-23,Wollondilly (A),382,NaN,NaN
61760,2021-07-23,Wollongong (C),1287,NaN,NaN
61761,2021-07-23,Woollahra (A),407,NaN,NaN


In [144]:
#fill NA (no cases) with zeros
cases_tests['num_cases'] = cases_tests['num_cases'].fillna(0)
cases_tests

,test_date,lga_name19,test_count,notification_date,num_cases
0,2020-01-01,Blacktown (C),1,NaN,0.0
1,2020-01-01,Camden (A),2,NaN,0.0
2,2020-01-01,Canada Bay (A),2,NaN,0.0
3,2020-01-01,Canterbury-Bankstown (A),2,NaN,0.0
4,2020-01-01,Cumberland (A),1,NaN,0.0
...,...,...,...,...,...
61758,2021-07-23,Wingecarribee (A),353,2021-07-23,1.0
61759,2021-07-23,Wollondilly (A),382,NaN,0.0
61760,2021-07-23,Wollongong (C),1287,NaN,0.0
61761,2021-07-23,Woollahra (A),407,NaN,0.0


In [148]:
#get the rate of cases per
cases_tests['positive_rate'] = cases_tests['num_cases']/cases_tests['test_count']
cases_tests

,test_date,lga_name19,test_count,notification_date,num_cases,positive_rate
0,2020-01-01,Blacktown (C),1,NaN,0.0,0.000000
1,2020-01-01,Camden (A),2,NaN,0.0,0.000000
2,2020-01-01,Canada Bay (A),2,NaN,0.0,0.000000
3,2020-01-01,Canterbury-Bankstown (A),2,NaN,0.0,0.000000
4,2020-01-01,Cumberland (A),1,NaN,0.0,0.000000
...,...,...,...,...,...,...
61758,2021-07-23,Wingecarribee (A),353,2021-07-23,1.0,0.002833
61759,2021-07-23,Wollondilly (A),382,NaN,0.0,0.000000
61760,2021-07-23,Wollongong (C),1287,NaN,0.0,0.000000
61761,2021-07-23,Woollahra (A),407,NaN,0.0,0.000000


In [151]:
#find the lga with highest postive rate
#cases_tests[(cases_tests['test_date']>'2021-06-01')&(cases_tests['test_count']>100)].sort_values(by='positive_rate', ascending=False)

#focus on cases/tests since June 2021
cases_tests[cases_tests['test_date']>='2021-06-01']

,test_date,lga_name19,test_count,notification_date,num_cases,positive_rate
60144,2021-07-11,Fairfield (C),1873,2021-07-11,74.0,0.039509
60020,2021-07-10,Fairfield (C),1499,2021-07-10,59.0,0.039360
61397,2021-07-21,Burwood (A),201,2021-07-21,4.0,0.019900
59488,2021-07-06,Burwood (A),130,2021-07-06,2.0,0.015385
59511,2021-07-06,Fairfield (C),866,2021-07-06,11.0,0.012702
...,...,...,...,...,...,...
57832,2021-06-23,Bayside (A),2438,NaN,0.0,0.000000
57836,2021-06-23,Blacktown (C),2070,NaN,0.0,0.000000
57839,2021-06-23,Blue Mountains (C),391,NaN,0.0,0.000000
57843,2021-06-23,Burwood (A),215,NaN,0.0,0.000000


In [163]:
#get the lga_name
for i in cases_tests.lga_name19.unique():
  print(i)

import numpy
numpy.savetxt("LGAs.csv", cases_tests.lga_name19.unique(), delimiter=",",fmt='%s')

Blacktown (C)
Camden (A)
Canada Bay (A)
Canterbury-Bankstown (A)
Cumberland (A)
Hunters Hill (A)
Inner West (A)
Ku-ring-gai (A)
Liverpool (C)
Parramatta (C)
Penrith (C)
Strathfield (A)
The Hills Shire (A)
Wollondilly (A)
Tweed (A)
Oberon (A)
Northern Beaches (A)
Bayside (A)
Burwood (A)
Georges River (A)
Fairfield (C)
Randwick (C)
Ryde (C)
Sydney (C)
Willoughby (C)
Bathurst Regional (A)
Campbelltown (C) (NSW)
Maitland (C)
Sutherland Shire (A)
Hornsby (A)
Newcastle (C)
Queanbeyan-Palerang Regional (A)
Bega Valley (A)
Blue Mountains (C)
Central Coast (C) (NSW)
Snowy Valleys (A)
Wagga Wagga (C)
North Sydney (A)
Armidale Regional (A)
Bourke (A)
Lake Macquarie (C)
Lismore (C)
Waverley (A)
Woollahra (A)
Hawkesbury (C)
Inverell (A)
Liverpool Plains (A)
Shoalhaven (C)
Singleton (A)
Wingecarribee (A)
Wollongong (C)
Bellingen (A)
Lane Cove (A)
Mosman (A)
Hilltops (A)
Port Stephens (A)
Balranald (A)
Byron (A)
Port Macquarie-Hastings (A)
Kempsey (A)
Shellharbour (C)
Cessnock (C)
Mid-Coast (A)
Tamwo

In [159]:
"string"[:-3]

'str'